## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-29-11-47-20 +0000,bbc,When is the Budget and what might be in it?,https://www.bbc.com/news/articles/cj4w44w42j5o...
1,2025-09-29-11-47-01 +0000,nyt,"Trump’s $100,000 Visa Upends Lives: ‘My Dreams...",https://www.nytimes.com/2025/09/29/business/in...
2,2025-09-29-11-46-19 +0000,nyt,"Typhoon Bualoi Slams Into Central Vietnam, Add...",https://www.nytimes.com/2025/09/29/world/asia/...
3,2025-09-29-11-44-36 +0000,bbc,"Rescue deal saves Claire's but 1,000 jobs rema...",https://www.bbc.com/news/articles/cder59gzx5xo...
4,2025-09-29-11-38-25 +0000,bbc,The Indian superstar whose political rally tur...,https://www.bbc.com/news/articles/c62lj9x5y60o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,30
69,new,13
125,shutdown,12
132,church,12
131,michigan,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
162,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,105
72,2025-09-29-09-00-00 +0000,wsj,Congressional leaders will head to the White H...,https://www.wsj.com/politics/policy/trump-face...,82
115,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...,75
127,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...,64
209,2025-09-28-14-41-11 +0000,nypost,Trump agrees to last-minute meeting with Jeffr...,https://nypost.com/2025/09/28/us-news/presiden...,62


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
162,105,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
127,64,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
189,54,2025-09-28-16-54-04 +0000,cbc,"4 dead, 8 wounded in shooting and fire at Mich...",https://www.cbc.ca/news/world/multiple-people-...
72,49,2025-09-29-09-00-00 +0000,wsj,Congressional leaders will head to the White H...,https://www.wsj.com/politics/policy/trump-face...
134,33,2025-09-28-23-26-36 +0000,nypost,Mississippi man shows up drunk to bail out pal...,https://nypost.com/2025/09/28/us-news/mississi...
50,28,2025-09-29-09-30-00 +0000,wsj,"The Pentagon, alarmed at the low weapons stock...",https://www.wsj.com/politics/national-security...
13,25,2025-09-29-11-14-42 +0000,bbc,Moldova's pro-EU party wins vote mired in clai...,https://www.bbc.com/news/articles/cx2rdlj8ejgo...
71,24,2025-09-29-09-00-00 +0000,wsj,"Rahm Emanuel, former Democratic congressman, W...",https://www.wsj.com/politics/policy/rahm-emanu...
8,23,2025-09-29-11-25-31 +0000,nypost,Teen runaway was likely dead for ‘several week...,https://nypost.com/2025/09/29/us-news/teen-run...
107,23,2025-09-29-03-50-42 +0000,nypost,Trump’s Portland troop plan sparks fierce lega...,https://nypost.com/2025/09/28/us-news/trumps-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
